In [2]:
import pandas as pd
import os 
import json
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials_gpe-analytics.json"




# The Data 



Note: 

> Tag = the tag applied (e.g. google etc) 
> Query = the product applied (e.g. headsets etc)



In [3]:
all_preds = pd.read_json('../data/predictions/final_preds/all_preds.json',orient='records',lines=True)

In [4]:
all_preds['Page Type'].unique()

array(['reddit', 'news', 'twitter', 'instagram'], dtype=object)

In [5]:
for x in list(all_preds.columns):
    print(x)

Unnamed: 0
Query Id
Query Name
Date
Title
text
Full Text
uid
Domain
Sentiment
Emotion
Page Type
Language
Country Code
Continent Code
Continent
Country
Region Code
Region
City Code
Account Type
Assignment
Author
Avatar
Category Details
Checked
City
Display URLs
Expanded URLs
Facebook Author ID
Facebook Comments
Facebook Likes
Facebook Role
Facebook Shares
Facebook Subtype
Full Name
Gender
Hashtags
Impact
Impressions
Instagram Comments
Instagram Followers
Instagram Following
Instagram Likes
Instagram Posts
Interest
Last Assignment Date
Latitude
Location Name
Longitude
Media Filter
Media URLs
Mentioned Authors
Priority
Professions
Resource Id
Short URLs
Starred
Status
Subtype
Tags
Thread Author
Thread Created Date
Thread Entry Type
Thread Id
Thread URL
Total Monthly Visitors
Twitter Author ID
Twitter Channel Role
Twitter Followers
Twitter Following
Twitter Reply Count
Twitter Reply to
Twitter Retweet of
Twitter Retweets
Twitter Likes
Twitter Tweets
Twitter Verified
Reach (new)
Reddit Scor

In [7]:
def all_preds_configuration(df):    
    # Change date column to timestamp
    if 'date' in df:
        df['date'] = df['date'].combine_first(df['Date'])
    else:
        df['date']  = df['Date']

    if 'author' in df:
        df['author'] = df['author'].combine_first(df['Author'])
    else:
        df['author'] = df['Author']

    # #Combine author for the news 
    df['author'] = df['author'].combine_first(df['Domain'])
        
    # # Now convert timestamp to string '2022-12-12'
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    # # Twitter
    df['Engagements'] = df.apply(lambda x: x['Twitter Likes'] + x['Twitter Retweets'],axis=1)
    # # Reddit
    if 'redditScore' in df:
       df['engagements'] = df['redditScore'].combine_first(df['Engagements'])
    else:
       df['engagements'] = df['Engagements']
    
    # # Create updated source column
    df['updated_source'] = df.apply(lambda x: x['Page Type'] if x['Page Type'] else 'reddit',axis=1)
    
    #df['link_url'] = df.apply(lambda x: x.uid if x.updated_source!='reddit' else x.url,axis=1)
    df['link_url'] = df['uid']
    bad_colums = [
        "Reach (new)", 
        "Google AR/VR: Companies (Parent - products)",
        "[Chrome AI Snap] Browsers",
        "[Chrome AI Snap] ChatGPT vs Bard",
        "[Chrome AI Snap] Search Engines",
        "Engagements",
        "Date",
        "Author",
        "Google AR/VR: Categories",
        "Google AR/VR: Companies",
        "Google AR/VR: Companies (Parent - products)",
        "Google AR/VR: Events",
        "Google AR/VR: Glasses",
        "Google AR/VR: Headsets"]
    df.drop(columns=bad_colums, inplace=True)
    
    return df 
    
    
    

In [8]:
all_preds = all_preds_configuration(all_preds)

In [9]:
# Create Table 
client = bigquery.Client()

In [11]:
# Create Table 
client = bigquery.Client()
# Define table name, in format dataset.table_name
table = 'gpe-analytics.project_google_arxr_analytics_20240508.ar_xr_predictions_update'

# Load data to BQ
job = client.load_table_from_dataframe(all_preds, table)

In [12]:
len(all_preds.loc[all_preds['author'].isnull()])

0

In [13]:
all_preds.loc[all_preds['uid']=='https://appleworld.today/archives/109141']['author']

Series([], Name: author, dtype: object)

In [14]:
for x in all_preds.columns:
    print(x+'\n')

Unnamed: 0

Query Id

Query Name

Title

text

Full Text

uid

Domain

Sentiment

Emotion

Page Type

Language

Country Code

Continent Code

Continent

Country

Region Code

Region

City Code

Account Type

Assignment

Avatar

Category Details

Checked

City

Display URLs

Expanded URLs

Facebook Author ID

Facebook Comments

Facebook Likes

Facebook Role

Facebook Shares

Facebook Subtype

Full Name

Gender

Hashtags

Impact

Impressions

Instagram Comments

Instagram Followers

Instagram Following

Instagram Likes

Instagram Posts

Interest

Last Assignment Date

Latitude

Location Name

Longitude

Media Filter

Media URLs

Mentioned Authors

Priority

Professions

Resource Id

Short URLs

Starred

Status

Subtype

Tags

Thread Author

Thread Created Date

Thread Entry Type

Thread Id

Thread URL

Total Monthly Visitors

Twitter Author ID

Twitter Channel Role

Twitter Followers

Twitter Following

Twitter Reply Count

Twitter Reply to

Twitter Retweet of

Twitter Retweets

Twitter 

In [15]:
['Impressions','Impact','reachEstimate','redditScore']

['Impressions', 'Impact', 'reachEstimate', 'redditScore']

In [17]:
all_preds_table = all_preds[['date','query','tag','llm_label','source','uid','text','Impressions','Impact']]

In [68]:
# Create Table 
client = bigquery.Client()
# Define table name, in format dataset.table_name
table = 'gpe-analytics.project_google_arxr_analytics_20240105.ar_xr_predictions_table'

# Load data to BQ
job = client.load_table_from_dataframe(all_preds_table, table)

# Do Clustering 

In [22]:
folder = 'gs://mm-gpe-data/project_google_arxr_analytics_20240508'

In [19]:
# Preprocessing keys 
{"text":"Would you prefer for Apple to adapt to the industry standard of RCS? #fyp #android #apple #iphone #samsung #google #ipager #phone #greenbubble #bluebubble #rcs #androidvsiphone #getthemessage","uid":"https:\/\/www.tiktok.com\/@redirect-to\/video\/7287566836936346912","source":"tiktok","date":"2024-01-04"}.keys()

<>:2: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\/'
/var/folders/q7/6w64406173s6jh0blpll135c0000gq/T/ipykernel_89916/3914621348.py:2: SyntaxWarning: invalid escape sequence '\/'
  {"text":"Would you prefer for Apple to adapt to the industry standard of RCS? #fyp #android #apple #iphone #samsung #google #ipager #phone #greenbubble #bluebubble #rcs #androidvsiphone #getthemessage","uid":"https:\/\/www.tiktok.com\/@redirect-to\/video\/7287566836936346912","source":"tiktok","date":"2024-01-04"}.keys()


dict_keys(['text', 'uid', 'source', 'date'])

In [6]:
# label map keys 
{"source": "Platform", "author": "NULL", "engagements": "NULL", "link": "Video_URL", "uid": "Video_URL", "date": "date", "queryName": "NULL", "query_title": "Video_Title", "subreddit": "NULL", "sentiment": "NULL"}.keys()

dict_keys(['source', 'author', 'engagements', 'link', 'uid', 'date', 'queryName', 'query_title', 'subreddit', 'sentiment'])

In [20]:
preprocessed_data = all_preds[['date','text','uid','source']]


In [21]:
preprocessed_data.to_json('preprocessed_data.json',orient='records',lines=True)

In [23]:
os.system(f'gsutil cp preprocessed_data.json {folder}/preprocessed/')

sh: gsutil: command not found


32512

In [25]:
all_preds.head()[['author','date']]

,author,date
0,saijanai,2024-02-09
1,machash.com,2024-03-28
2,RidgeMinecraft,2024-03-24
3,Julian Solanki,2024-01-22
4,BrainXchange,2024-02-28


In [26]:
all_preds.head()['url']

KeyError: 'url'

In [30]:
label_map = {'source':'source',
            'author':'author',
            'engagements':'NULL',
            'link':'NULL',
            'uid':'uid',
            'date':'date',
            'queryName':'NULL',
            'query_title':'NULL',
            'subreddit':'NULL',
            'sentiment':'NULL'}


with open('data_label_map2.json','w') as f:
    json.dump(label_map,f)
    
os.system(f'gsutil cp data_label_map2.json {folder}/label_maps/')


Copying file://data_label_map2.json [Content-Type=application/json]...
/ [1 files][  195.0 B/  195.0 B]                                                
Operation completed over 1 objects/195.0 B.                                      


0

In [105]:
# all_preds.to_json('raw_data.json',orient='records',lines=True)
os.system(f'gsutil cp raw_data.json {folder}/raw/')


Copying file://raw_data.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][  1.1 GiB/  1.1 GiB]   24.1 MiB/s                                   
Operation completed over 1 objects/1.1 GiB.                                      


0

In [ ]:
"""SELECT 
    * 
    FROM 
    (
        SELECT 
        DISTINCT LEFT(text, 300) AS text, 
        uid, 
        category_1, 
        probability_1, 
        category_2, 
        probability_2, 
        subreddit_category_1,
        subreddit_probability_1
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240109.redditnlp_output`
    )  
    LEFT JOIN (SELECT source ,uid,date engagements as NULL from gpe-analytics.project_google_arxr_analytics_20240109.raw_data) USING (uid)
    LEFT JOIN (
        SELECT 
        * 
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240109.clusters`
    ) USING (uid) 
    LEFT JOIN (SELECT * FROM `gpe-analytics.project_google_arxr_analytics_20240109.custom_nlp_tags`) USING (uid) LEFT JOIN (SELECT text as raw_text, uid,  FROM `gpe-analytics.project_google_arxr_analytics_20240109.raw_data`) USING (uid)"""

In [ ]:
CREATE TABLE `gpe-analytics.project_google_arxr_analytics_20240109.view_dashboard_final_update` as SELECT 
    * 
    FROM 
    (
        SELECT 
        DISTINCT LEFT(text, 300) AS text, 
        uid, 
        category_1, 
        probability_1, 
        category_2, 
        probability_2, 
        subreddit_category_1,
        subreddit_probability_1
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240109.redditnlp_output`
    )  
    LEFT JOIN (SELECT updated_source ,uid,date_,query, tag, llm_label, engagements_ from `gpe-analytics.project_google_arxr_analytics_20240109.ar_xr_predictions_table_update`) USING (uid)
    LEFT JOIN (
        SELECT 
        * 
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240109.clusters`
    ) USING (uid) 
    LEFT JOIN (SELECT * FROM `gpe-analytics.project_google_arxr_analytics_20240109.custom_nlp_tags`) USING (uid) 

# 2024/01/16 Create Clustering Dashboard 

In [107]:
all_preds = pd.read_json('../data/predictions/final_preds/all_preds.json',orient='records',lines=True)

In [111]:
for x in all_preds.columns:
    print(x)

Unnamed: 0
Query Id
Query Name
Date
Title
text
uid
Domain
Sentiment
Emotion
Page Type
Language
Country Code
Continent Code
Continent
Country
Region Code
Region
City Code
Account Type
Assignment
Author
Avatar
Category Details
Checked
City
Display URLs
Expanded URLs
Facebook Author ID
Facebook Comments
Facebook Likes
Facebook Role
Facebook Shares
Facebook Subtype
Full Name
Gender
Hashtags
Impact
Impressions
Instagram Comments
Instagram Followers
Instagram Following
Instagram Likes
Instagram Posts
Interest
Last Assignment Date
Latitude
Location Name
Longitude
Media Filter
Media URLs
Mentioned Authors
Priority
Professions
Resource Id
Short URLs
Starred
Status
Subtype
Tags
Thread Author
Thread Created Date
Thread Entry Type
Thread Id
Thread URL
Total Monthly Visitors
Twitter Author ID
Twitter Channel Role
Twitter Followers
Twitter Following
Twitter Reply Count
Twitter Reply to
Twitter Retweet of
Twitter Retweets
Twitter Likes
Twitter Tweets
Twitter Verified
Reach (new)
Reddit Score
Reddit S

In [ ]:
all_preds['twitter_engagements']